In [6]:
# the access key
client_id = "fd156f449b1a44359bb23cf0c2d67dfc"
client_secret = "3a69b492b52b4624b60b620b39e8d4ea"

In [7]:
# Code to get access token 

import requests
import base64

def get_access_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_header = {
        'Authorization': f'Basic {base64.b64encode((client_id + ":" + client_secret).encode()).decode()}'
    }
    auth_data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret
    }
    auth_response = requests.post(auth_url, data=auth_data, headers=auth_header)
    auth_response_data = auth_response.json()
    access_token = auth_response_data['access_token']
    return access_token


In [17]:
def get_genres(access_token, limit=50, offset=0):
    genres_url = 'https://api.spotify.com/v1/browse/categories'
    headers = {
        'Authorization': 'Bearer ' + access_token
    }
    params = {
        'limit': limit,
        'offset': offset
    }
    response = requests.get(genres_url, headers=headers, params=params)
    genres_data = response.json()
    genres = [(category['id'], category['name']) for category in genres_data['categories']['items']]
    return genres


In [18]:
access_token = get_access_token(client_id, client_secret)  
genres = get_genres(access_token)  

print("List of genres:")
for genre_id, genre_name in genres:
    print(genre_name) 


List of genres:
Made For You
New Releases
Summer
Country
Hip-Hop
Pop
Charts
GLOW
Black Music Month
Latin
Student
Discover
Rock
Dance/Electronic
In the car
Mood
Indie
R&B
Christian & Gospel
Workout
Disney
Música Mexicana
K-pop
Chill
Netflix
Sleep
Party
At Home
Decades
Love
Metal
Spotify CLASSICS
Jazz
Trending
Frequency
Classical
Folk & Acoustic
Focus
Soul
Kids & Family
Gaming
Anime
TV & Movies
Instrumental
Punk
Ambient
Blues
Cooking & Dining
Alternative
Travel


In [19]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = "fd156f449b1a44359bb23cf0c2d67dfc"
client_secret = "3a69b492b52b4624b60b620b39e8d4ea"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_artist_genres(artist_id):
    artist = sp.artist(artist_id)
    return artist['genres']

access_token = get_access_token(client_id, client_secret)  
def get_artist_genre(artist_id):
    artist = sp.artists(artist_id)
    return artist['genres']